# Experiments with Schemes for Exponential Decay
**Hans Petter Langtangen** (email: `hpl@simula.no`), Center for Biomedical Computing, Simula Research Laboratory and Department of Informatics, University of Oslo.

**May 21, 2015**

**Summary.** This report investigates the accuracy of three finite difference
schemes for the ordinary differential equation $u'=-au$ with the
aid of numerical experiments. Numerical artifacts are in particular
demonstrated.










## Mathematical problem
<div id="math:problem"></div>



We address the initial-value problem

<!-- Equation labels as ordinary links -->
<div id="ode"></div>

$$
\begin{equation}
u'(t) = -au(t), \quad t \in (0,T], \label{ode} \tag{1}
\end{equation}
$$

<!-- Equation labels as ordinary links -->
<div id="initial:value"></div>

$$
\begin{equation} 
u(0)  = I,                         \label{initial:value} \tag{2}
\end{equation}
$$

where $a$, $I$, and $T$ are prescribed parameters, and $u(t)$ is
the unknown function to be estimated. This mathematical model
is relevant for physical phenomena featuring exponential decay
in time, e.g., vertical pressure variation in the atmosphere,
cooling of an object, and radioactive decay.

## Numerical solution method
<div id="numerical:problem"></div>



We introduce a mesh in time with points $0 = t_0 < t_1 \cdots < t_{N_t}=T$.
For simplicity, we assume constant spacing $\Delta t$ between the
mesh points: $\Delta t = t_{n}-t_{n-1}$, $n=1,\ldots,N_t$. Let
$u^n$ be the numerical approximation to the exact solution at $t_n$.

The $\theta$-rule [[Iserles_2009]](#Iserles_2009)
is used to solve [(1)](#ode) numerically:

$$
u^{n+1} = \frac{1 - (1-\theta) a\Delta t}{1 + \theta a\Delta t}u^n,
$$

for $n=0,1,\ldots,N_t-1$. This scheme corresponds to

  * The [Forward Euler](http://en.wikipedia.org/wiki/Forward_Euler_method)
    scheme when $\theta=0$

  * The [Backward Euler](http://en.wikipedia.org/wiki/Backward_Euler_method)
    scheme when $\theta=1$

  * The [Crank-Nicolson](http://en.wikipedia.org/wiki/Crank-Nicolson)
    scheme when $\theta=1/2$

## Implementation


The numerical method is implemented in a Python function
[[Langtangen_2014]](#Langtangen_2014) `solver` (found in the [`model`](https://github.com/hplgit/INF5620/blob/gh-pages/src/decay/experiments/dc_mod.py) module):

In [1]:
def solver(I, a, T, dt, theta):
    """Solve u'=-a*u, u(0)=I, for t in (0,T] with steps of dt."""
    dt = float(dt)            # avoid integer division
    Nt = int(round(T/dt))     # no of time intervals
    T = Nt*dt                 # adjust T to fit time step dt
    u = zeros(Nt+1)           # array of u[n] values
    t = linspace(0, T, Nt+1)  # time mesh

    u[0] = I                  # assign initial condition
    for n in range(0, Nt):    # n=0,1,...,Nt-1
        u[n+1] = (1 - (1-theta)*a*dt)/(1 + theta*dt*a)*u[n]
    return u, t

def exact_solution(t, I, a):
    return I*exp(-a*t)

def explore(I, a, T, dt, theta=0.5):
    """
    Run a case with the solver, compute error measure,
    and plot the numerical and exact solutions (if makeplot=True).
    """
    u, t = solver(I, a, T, dt, theta)    # Numerical solution
    u_e = exact_solution(t, I, a)
    e = u_e - u
    E = sqrt(dt*sum(e**2))

    figure()                         # create new plot
    t_e = linspace(0, T, 1001)       # very fine mesh for u_e
    u_e = exact_solution(t_e, I, a)
    plot(t,   u,   'r--o')           # dashed red line with circles
    plot(t_e, u_e, 'b-')             # blue line for u_e
    legend(['numerical', 'exact'])
    xlabel('t')
    ylabel('u')
    title('Method: theta-rule, theta=%g, dt=%g' % (theta, dt))
    theta2name = {0: 'FE', 1: 'BE', 0.5: 'CN'}
    savefig('%s_%g.png' % (theta2name[theta], dt), dpi=150)
    savefig('%s_%g.pdf' % (theta2name[theta], dt))
    #show()  # run in batch
    return E

def define_command_line_options():
    import argparse
    parser = argparse.ArgumentParser()
    parser.add_argument('--I', '--initial_condition', type=float,
                        default=1.0, help='initial condition, u(0)',
                        metavar='I')
    parser.add_argument('--a', type=float,
                        default=1.0, help='coefficient in ODE',
                        metavar='a')
    parser.add_argument('--T', '--stop_time', type=float,
                        default=1.0, help='end time of simulation',
                        metavar='T')
    parser.add_argument('--dt', '--time_step_values', type=float,
                        default=[1.0], help='time step values',
                        metavar='dt', nargs='+', dest='dt_values')
    return parser

def read_command_line():
    parser = define_command_line_options()
    args = parser.parse_args()
    return args.I, args.a, args.T, args.dt_values

def main():
    """Conduct experiments with theta and dt values."""
    I, a, T, dt_values = read_command_line()
    for theta in 0, 0.5, 1:
        for dt in dt_values:
            E = explore(I, a, T, dt, theta)
            print '%3.1f %6.2f: %12.3E' % (theta, dt, E)

if __name__ == '__main__':
    main()


## Numerical experiments



We define a set of numerical experiments where $I$, $a$, and $T$ are
fixed, while $\Delta t$ and $\theta$ are varied.
In particular, $I=1$, $a=2$, $\Delta t = 1.25, 0.75, 0.5, 0.1$.



### The Backward Euler method





<p></p>
<img src="BE.png" width=800>





### The Crank-Nicolson method





<p></p>
<img src="CN.png" width=800>





### The Forward Euler method





<p></p>
<img src="FE.png" width=800>





### Error vs $\Delta t$



How $E$ varies with $\Delta t$ for $\theta=0,0.5,1$
is shown in [Figure](#fig:error).


**Observe:**

The data points for the three largest $\Delta t$ values in the
Forward Euler method are not relevant as the solution behaves
non-physically.



<div id="fig:error"></div>

<p>Variation of the error with the time step.</p>
<img src="error.png" width=400>




The numbers corresponding to the figure above are given in the table below.

<table border="1">
<thead>
<tr><th align="center">$\Delta t$</th> <th align="center">$\theta=0$</th> <th align="center">$\theta=0.5$</th> <th align="center">$\theta=1$</th> </tr>
</thead>
<tbody>
<tr><td align="right">   1.25          </td> <td align="right">   7.4630        </td> <td align="right">   0.2161          </td> <td align="right">   0.2440        </td> </tr>
<tr><td align="right">   0.75          </td> <td align="right">   0.6632        </td> <td align="right">   0.0744          </td> <td align="right">   0.1875        </td> </tr>
<tr><td align="right">   0.50          </td> <td align="right">   0.2797        </td> <td align="right">   0.0315          </td> <td align="right">   0.1397        </td> </tr>
<tr><td align="right">   0.10          </td> <td align="right">   0.0377        </td> <td align="right">   0.0012          </td> <td align="right">   0.0335        </td> </tr>
</tbody>
</table>

**Summary.**

1. $\theta =1$: $E\sim \Delta t$ (first-order convergence).

2. $\theta =0.5$: $E\sim \Delta t^2$ (second-order convergence).

3. $\theta =1$ is always stable and gives qualitatively corrects results.

4. $\theta =0.5$ never blows up, but may give oscillating solutions
   if $\Delta t$ is not sufficiently small.

5. $\theta =0$ suffers from fast-growing solution if $\Delta t$ is
   not small enough, but even below this limit one can have oscillating
   solutions (unless $\Delta t$ is sufficiently small).



## Bibliography


 1. <div id="Iserles_2009"></div> **A. Iserles**. 
    *A First Course in the Numerical Analysis of Differential Equations*,
    second edition,
    Cambridge University Press,
    2009.

 2. <div id="Langtangen_2014"></div> **H. P. Langtangen**. 
    *A Primer on Scientific Programming With Python*,
    fourth edition,
    Springer,
    2014.